In [1]:
import cv2
import random
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def imgCrop(img, x_start, x_end, y_start, y_end):
    img_crop = img[x_start : x_end, y_start : y_end]
    return img_crop

In [3]:
def imgColorShift(img, b_shift, g_shift, r_shift):
    B, G, R = cv2.split(img)
    if b_shift == 0:
        pass
    elif b_shift > 0:
        lim = 255 - b_shift
        B[B > lim] = 255
        B[B <= lim] = (b_shift + B[B <= lim]).astype(img.dtype)
        
    if g_shift == 0:
        pass
    elif g_shift > 0:
        lim = 255 - g_shift
        G[G > lim] = 255
        G[G <= lim] = (g_shift + G[G <= lim]).astype(img.dtype)  
        
    if r_shift == 0:
        pass
    elif r_shift > 0:
        lim = 255 - r_shift
        R[R > lim] = 255
        R[R <= lim] = (r_shift + R[R <= lim]).astype(img.dtype)    
    img_merge = cv2.merge((B, G, R))
    return img_merge

In [4]:
def imgSimilarityTransform(img, angle, scale, center_x, center_y):
    M = cv2.getRotationMatrix2D((center_x, center_y), angle, scale)
    img_rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    return img_rotated

In [5]:
def imgPerspectiveTransform(img, random_margin):
    height, width, channels = img.shape
    # warp:
    x1 = random.randint(-random_margin, random_margin)
    y1 = random.randint(-random_margin, random_margin)
    x2 = random.randint(width - random_margin - 1, width - 1)
    y2 = random.randint(-random_margin, random_margin)
    x3 = random.randint(width - random_margin - 1, width - 1)
    y3 = random.randint(height - random_margin - 1, height - 1)
    x4 = random.randint(-random_margin, random_margin)
    y4 = random.randint(height - random_margin - 1, height - 1)

    dx1 = random.randint(-random_margin, random_margin)
    dy1 = random.randint(-random_margin, random_margin)
    dx2 = random.randint(width - random_margin - 1, width - 1)
    dy2 = random.randint(-random_margin, random_margin)
    dx3 = random.randint(width - random_margin - 1, width - 1)
    dy3 = random.randint(height - random_margin - 1, height - 1)
    dx4 = random.randint(-random_margin, random_margin)
    dy4 = random.randint(height - random_margin - 1, height - 1)

    pts1 = np.float32([[x1, y1], [x2, y2], [x3, y3], [x4, y4]])#做投射变换需要4个点，原来的点
    pts2 = np.float32([[dx1, dy1], [dx2, dy2], [dx3, dy3], [dx4, dy4]])#做投射变换需要4个点，目标点
    M_warp = cv2.getPerspectiveTransform(pts1, pts2)
    img_warp = cv2.warpPerspective(img, M_warp, (width, height))
    return img_warp

In [6]:
def imgDataAugmentation(img, crop, shift, center, angle, scale, warp_value):
    img_crop = imgCrop(img,*crop)
    img_color_shift = imgColorShift(img_crop, *shift)
    img_rotated = imgSimilarityTransform(img_color_shift, angle, scale, *center)
    img_warp = imgPerspectiveTransform(img_rotated,warp_value)
    return img_warp
    

In [7]:
img_input = cv2.imread('C:/Lenna.png')
cv2.imshow('Input Image', img_input)
#img_input, crop, color_shift(bgr), center(x,y), angle, scale, warp_value 
img_output = imgDataAugmentation(img_input, [0,512,0,512], [-20,20,-20], (img_input.shape[1] / 2, img_input.shape[0] / 2), 30, 1, 60)
cv2.imshow('Output Image', img_output)
key = cv2.waitKey(0)
if key == 27:
    cv2.destroyAllWindows()
